In [2]:
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import xgboost as xgb

In [188]:
for l in open('News_Category_Dataset.json'):
    print l
    break

{"short_description": "She left her husband. He killed their children. Just another day in America.", "headline": "There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV", "date": "2018-05-26", "link": "https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89", "authors": "Melissa Jeltsen", "category": "CRIME"}



In [10]:
def readjson(f):
  for l in open(f):
    yield eval(l)

In [90]:
data=readjson('News_Category_Dataset.json')
data=list(data)

In [12]:
wordCount = defaultdict(int)
for d in data:
  for w in d['short_description'].split():
    wordCount[w] += 1

In [16]:
print len(wordCount)

138027


In [17]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stemmer = PorterStemmer()
for d in data:
  r = ''.join([c for c in d['short_description'].lower() if not c in punctuation])
  for w in r.split():
    #w = stemmer.stem(w) # with stemming
    wordCount[w] += 1

In [34]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
  r = ''.join([c for c in d['short_description'].lower() if not c in punctuation])
  for w in r.split():
    wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [35]:
words = [x[1] for x in counts[:1000]]

In [83]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
  feat = [0]*len(words)
  r = ''.join([c for c in datum['short_description'].lower() if not c in punctuation])
  for w in r.split():
    if w in words:
      feat[wordId[w]] += 1
  feat.append(1) #offset
  return feat


In [91]:
X = [feature(d) for d in data]
y = [d['category'] for d in data]

In [92]:
len(y)

124989

In [93]:
len(X)

124989

In [95]:
len(list(set(y)))

31

In [114]:
def label2onehot(lbl):
    d = numpy.zeros(( max(lbl) + 1, len(lbl) ))
    d[lbl, numpy.arange(0, len(lbl) )] = 1
    return d

In [100]:
ylabel_id = dict(zip(list(set(y)), range(31)) )

In [101]:
ylabel_id

{'ARTS': 7,
 'ARTS & CULTURE': 19,
 'BLACK VOICES': 25,
 'BUSINESS': 0,
 'COLLEGE': 27,
 'COMEDY': 12,
 'CRIME': 2,
 'EDUCATION': 29,
 'ENTERTAINMENT': 1,
 'FIFTY': 8,
 'GOOD NEWS': 15,
 'GREEN': 28,
 'HEALTHY LIVING': 30,
 'IMPACT': 6,
 'LATINO VOICES': 4,
 'MEDIA': 16,
 'PARENTS': 3,
 'POLITICS': 5,
 'QUEER VOICES': 21,
 'RELIGION': 9,
 'SCIENCE': 11,
 'SPORTS': 23,
 'STYLE': 14,
 'TASTE': 24,
 'TECH': 10,
 'THE WORLDPOST': 22,
 'TRAVEL': 20,
 'WEIRD NEWS': 17,
 'WOMEN': 13,
 'WORLD NEWS': 18,
 'WORLDPOST': 26}

In [175]:
dtrain=[ int(ylabel_id[d])  for d in y ]

In [115]:
dtrain = label2onehot(ytrain)

In [162]:
max(ytrain)

30

In [133]:
dtrain=dtrain.T

In [141]:
dtrain.shape

(124989, 31)

In [142]:
X_all.shape

(124989, 1001)

In [178]:
X_all=numpy.array(X)
dtrain=numpy.array(dtrain)

In [179]:
Xtrain=X_all[0:int(0.8*len(X_all))]
Xtest=X_all[int(0.8*len(X_all)):]

In [180]:
ytrain=dtrain[0:int(0.8*len(dtrain))]
ytest=dtrain[int(0.8*len(dtrain)):]

In [181]:
print Xtrain.shape
print Xtest.shape
print ytrain.shape
print ytest.shape

(99991, 1001)
(24998, 1001)
(99991,)
(24998,)


In [182]:
xg_train = xgb.DMatrix(Xtrain, label=ytrain)
xg_test = xgb.DMatrix(Xtest, label=ytest)

In [185]:
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 31

In [186]:
num_round
param

{'eta': 0.1,
 'max_depth': 6,
 'nthread': 4,
 'num_class': 31,
 'objective': 'multi:softmax',
 'silent': 1}

In [187]:
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 100
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
pred = bst.predict(xg_test)
error_rate = numpy.sum(pred != ytest) / ytest.shape[0]
print('Test error using softmax = {}'.format(error_rate))

[0]	train-merror:0.666750	test-merror:0.817705
[1]	train-merror:0.662220	test-merror:0.817305
[2]	train-merror:0.659349	test-merror:0.815425
[3]	train-merror:0.657389	test-merror:0.815265
[4]	train-merror:0.655749	test-merror:0.814505
[5]	train-merror:0.653969	test-merror:0.812145
[6]	train-merror:0.651889	test-merror:0.811505
[7]	train-merror:0.651469	test-merror:0.811625
[8]	train-merror:0.651169	test-merror:0.811425
[9]	train-merror:0.649738	test-merror:0.811185
[10]	train-merror:0.648918	test-merror:0.810545
[11]	train-merror:0.647898	test-merror:0.810825
[12]	train-merror:0.647268	test-merror:0.810505
[13]	train-merror:0.646098	test-merror:0.810585
[14]	train-merror:0.645828	test-merror:0.809945
[15]	train-merror:0.644788	test-merror:0.809025
[16]	train-merror:0.644128	test-merror:0.808905
[17]	train-merror:0.643458	test-merror:0.808545
[18]	train-merror:0.642678	test-merror:0.808345
[19]	train-merror:0.642268	test-merror:0.807745
[20]	train-merror:0.641488	test-merror:0.807425
[2

Test error using softmax = 0


[99]	train-merror:0.598754	test-merror:0.787583


{"short_description": "She left her husband. He killed their children. Just another day in America.", "headline": "There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV", "date": "2018-05-26", "link": "https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89", "authors": "Melissa Jeltsen", "category": "CRIME"}

